# Import the modules you will need


In [1]:
# modules need for working with date and time
from datetime import date
from dateutil.relativedelta import relativedelta
import time

# Modules need for webscraping
from bs4 import BeautifulSoup
import requests
import json

# modules needed for working with DataFrames and Excel
import pandas as pd
from openpyxl import load_workbook

# Needed to remove warnings from jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# Change User-Agent in requests headers

In [2]:
# update the "User-Agent" for you header otherwise the sec will reject your
# requests and you will get a 403 error
headers = requests.utils.default_headers()
headers.update({"User-Agent": "insert name here, enter email",})

# Put in the inputs of what you are looking for


In [3]:
# input the companies ticker and whether you want their 10Ks or 10Qs
search_for_ticker = input("What companies cik number are you looking for: input ticker ")

# maybe put in what time frame you are looking for
search_through_files_years_back = int(input('How many years of filings do you want? '))
#ten_ks_or_qs = input('Do you want the Ks or the Qs') #impliment later

type_of_filing = input('Are you looking for 10-K, or 10-Q filings? ')

What companies cik number are you looking for: input ticker AAPL
How many years of filings do you want? 2
Are you looking for 10-K, or 10-Q filings? 10-K


# Set the dates you are grabbing files for

In [4]:
todays_date = date.today()
till_date = todays_date - relativedelta(years=search_through_files_years_back)
todays_date = str(todays_date).replace('-','/')
till_date = str(till_date).replace('-','/')

# Send the SEC a Request for all their company tickers and CIK numbers

In [5]:
# below is the url where all the sec tickers are found
url_of_tickers = "https://www.sec.gov/files/company_tickers.json"

# send SEC the page request
page = requests.get(url_of_tickers, headers = headers)
json_ticker_dictionary = page.json()

# Put the Data into a pandas DataFrame

In [6]:
# create a DataFrame with your ticker dictionary
df = pd.DataFrame(json_ticker_dictionary)

# transpose the data frame so it is easier to view
df = df.transpose()

# set the index to the ticker column that way it's easy to search for
# and return the cik number
df.set_index(keys = 'ticker', inplace = True)

# search for the ticker and store the cik in a var
cik_num = str(df.loc[search_for_ticker, 'cik_str'])

# Send the SEC a Request to get data on all the company filings

In [7]:
# creating a temporary string because the SEC has the company files.json down
url_of_company_filings = f"https://www.sec.gov/Archives/edgar/data/320193/index.json"

# I don't know why i set this request up different but it works
page = requests.get(url_of_company_filings, headers = headers).text
accession_number = json.loads(page)

# Create a DataFrame with all the company filings

In [8]:
# put the dictionary of 'items' aka the filings into a DataFrame
df = pd.DataFrame(accession_number['directory']['item'])

# create a column with of the url of every indiviudal filing folder 
df["Url"] = f"https://www.sec.gov/Archives/edgar/data/{cik_num}/" + df["name"] + "/index.json"

# create a temp url
base_url = f"https://www.sec.gov/Archives/edgar/data/{cik_num}/"
#df['Url'] = "https://www.sec.gov/Archives/edgar/data/320193/" + df['name'] + '/index.json'

# eleminate columns that aren't needed. This is an unnecessary step
df.drop(["type","size"], axis= "columns", inplace= True)

# rename the last-modified column to date
df.rename(columns={"last-modified":"Date", "name":"Accession #"}, inplace=True)

# split the date and time into seperate columns and remove the time column
df[['Date', 'Time']] = df['Date'].str.split(" ", n=1, expand = True)
df.drop(['Time'], axis="columns", inplace=True) # if programs get more advanced you can bring back the time

# Filter the DataFrame to return the filings between a given dates

In [9]:
# Put the Date column into datetime format
df['Date'] = pd.to_datetime(df['Date'])

# only return the filings urls between cetain dates
mask = (df['Date'] <= todays_date) & (df['Date'] >= till_date)
df = df[mask]

# create an integer index of data frame and remove the index column
df.reset_index(inplace= True)
df.drop('index', axis= 1, inplace= True)

# Loop through the DataFrame, create a Dataframe to return the '.txt' filing

In [10]:
# Create a list to store store the filings that are 10q, 10k or 8k etc.
# It is in the filings fold which is the url created in the DataFrame above.
# Which will be put into a data frame later.
filings_list = []

# loop through the filing folders
for i in range(len(df.index)):
    
    # store the filings fold url
    filings_folder_url = df.iloc[i,2]
    
    # request the SEC for filing folders page and store data into dictionary
    page = requests.get(filings_folder_url, headers=headers).text
    folder_filing_dictionary = json.loads(page)
    
    # create a DataFrame with the filiing folder information
    filing_df = pd.DataFrame(folder_filing_dictionary['directory']['item'])
    
    # return the filing ending with '.txt'
    filings_list.append(filing_df.iloc[2])

    # the sec can only accept 10 request a second so sleep it for .10 or .11 secs
    time.sleep(.11)
    
    # the code below is to see if you're getting the '.txt' filing
#    print(filing_df.iloc[2])

# Create a DataFrame to store all the filings with '.txt'

In [11]:
# create a DataFrame named fdf aka filings DataFrame
fdf = pd.DataFrame(filings_list, columns=["last-modified", "name", "type", "size"])

# reset your index
fdf = fdf.reset_index(level=0)

# remove all the uneccessary columns
fdf.drop(['last-modified', 'type', 'size', 'index'],axis=1, inplace= True)

# Merge your Filings DataFrame with your other DataFrame

In [12]:
# merge your fdf to your df
df = pd.merge(df, fdf, left_index= True, right_index= True)

# convert the column types from objects to a string
df['Url'] = df['Url'].astype('string')
df['name'] = df['name'].astype('string')

# remove the 'index.json'
df['Url'] = df['Url'].str.replace('index.json', '')

# create a new column of the individual filing urls ending in '.txt'
df['Filing_urls'] = df['Url'] + df['name']

# Loop through DataFrame to find yearly (10ks) (pending.. or quarterly reports (10qs)

In [13]:
tenk_dictionary = {}

for i in range(len(df.index)):
    # loop through your filings column send a page request to the SEC
    url = df['Filing_urls'][i]
    page = requests.get(url, headers = headers).text
    doc = BeautifulSoup(page, 'lxml')
    
    # find the filing type
    filing_type = doc.type.find(text= True, recursive = False).strip()
    
    # create a dictionary with all the urls and accession numbers 
    # of your filing type you chose
    if filing_type == type_of_filing:     
#         print(url)
        # THIS MIGHT BE WHERE YOU PUT THE ACCESSION NUMBER
        tenk_dictionary[df['Date'][i]] = df['Url'][i], df['Accession #'][i]
    
    # SEC only accepts 10 requests a second so make sure you sleep the loop.
    time.sleep(.12)

# Create DataFrame with FilingSummary.xml Url

In [14]:
df = pd.DataFrame(tenk_dictionary.items(), columns= ['Date', 'Url'])

# Clean your data
# and add column for FilingSummary.xml
df[['Url','Accession #']] = df['Url'].tolist()
df['Url'] = df['Url'].astype('string')
df['Accession #'] = df['Accession #'].astype('string')
df['Filing.xml_url'] =  df['Url'] + 'FilingSummary.xml'

# Heading

In [15]:
# Append to this list all to a dictionary outside the for loop
dictionary_lists = []

for i in range(len(df.index)):
    
    # create the basis for your dictionary
    master_dictionary_of_all_tables = {}
    saucey_short_name_list = []
    saucey_html_file_name_list = []
    
    # loop through your filings column send a page request to the SEC
    url = df['Filing.xml_url'][i]
    xml_page = requests.get(url, headers = headers)
    sauce = BeautifulSoup(xml_page.content, 'lxml')
    bod = sauce.find('body')
    
    # I cannot get this to run without an Attribution error
    # just except and move to the next thing
    try:
        for link in bod.find_all('report'):
            # insert data into your lists
            saucey_short_name_list.append(link.find('shortname').string)
            saucey_html_file_name_list.append(link.find('htmlfilename').string)
            
            #(code) the code below might stop the attribution error if put here
            
            # Use the list above to create your dictionary
            master_dictionary_of_all_tables[str(df['Date'][i])] = saucey_short_name_list
            master_dictionary_of_all_tables[df['Accession #'][i]] = saucey_html_file_name_list
            
    except AttributeError:
        print('Attribution Error: carry on!')
        next
        
    #(code) this code might need to be placed right after the list     
    while len(saucey_short_name_list) != len(saucey_html_file_name_list):
        saucey_short_name_list.pop()
    
    # apped all your dictionaries to a list
    dictionary_lists.append(master_dictionary_of_all_tables)

Attribution Error: carry on!
Attribution Error: carry on!


# Turn the dictionary list into a DataFrame list

In [16]:
dataframe_list = []

for i in range(len(dictionary_lists)):
    df = pd.DataFrame(dictionary_lists[i])
    dataframe_list.append(df)

# Loop through DataFramesList, rename the first column to 'ShortName' and then merg all the DataFrames in the DataFramesList

In [17]:
# Change first column to 'ShortName'
for i in range(len(dataframe_list)):
    dataframe_list[i].rename({dataframe_list[i].columns[0]:'ShortName'}, axis=1, inplace=True)
    dataframe_list[i].set_index('ShortName', inplace=True)
    
# merge all DataFrames into one.
for i in range(len(dataframe_list)-1):
    dataframe_list[0] = dataframe_list[0].join(dataframe_list[i+1],  how='left')


In [18]:
# Make a new DF that's = to the first item in dataframe list
filing_df = dataframe_list[0]
len_of_filing_df = len(filing_df.columns)

# create column of the urls you're going to scrape tables for
for i in range(len_of_filing_df):
    filing_df['Url' + str(i)] =  base_url + filing_df.columns[i] + '/' + filing_df[filing_df.columns[i]]
    
# drop unnecessary columns    
filing_df.drop(filing_df.columns[0:len_of_filing_df], axis= 1, inplace= True)

In [19]:
# select data you want to grab, income statement, balance sheet, cash flows, etc.
filing_df = filing_df.iloc[[1,3,6]]

In [20]:
# list will hold all the data frames that you will be joining
list_page_lists_requests = []

#df_list = []
for x in range(len(filing_df.index)):
    page_list_requests = []
    for i in filing_df.columns:
        
        # requests data
        page = requests.get(filing_df[i][x], headers = headers).text
        # put into a list
        page_list_requests.append(page)
    # put the list above into a list every individual list will
    # be put into a DataFrame
    list_page_lists_requests.append(page_list_requests)
    

In [21]:
# put all DataFrames into this list
lists_of_df_lists = []

for x in range(len(list_page_lists_requests)):
    df_list = []
    
    for i in list_page_lists_requests[x]:
        
        # read data into a DataFrame
        df = pd.read_html(i)
        df = df[0]
        
        # if DF is a multileve index remove it
        if isinstance(df.columns, pd.core.indexes.multi.MultiIndex):
            df.columns = df.columns.droplevel(0)
        
        # rename lineitems column to ledger and set it as the index 
        df.rename({df.columns[0]:'Ledger'}, axis=1, inplace= True)
        df.set_index('Ledger', inplace= True)
        
        # set df to string to remove unwanted characters
        df = df.astype('string')
        
        # clean data so you can convert DF astype to float
        for i in df.columns.to_list():
            df[i] = df[i].str.replace(',','')
            df[i] = df[i].str.replace('$','')
            df[i] = df[i].str.replace(')','')
            df[i] = df[i].str.replace('(','-')

        df = df.astype('float')
        
        # make sure index has unique values otherwise
        # they will not join properly it adds numbers to any
        # duplicate line items
        df_ledger = df.index.to_list()
        
        UniqueValues = set(df_ledger)

        for y in UniqueValues:
            number = 0
            
            for z in range(0,len(df_ledger)):
                if df_ledger[z] == y:
                    number += 1
                    if number >= 2:
                        df_ledger[z] += str(number)
                              
        # set the new index with no duplicate items                
        df.set_index(pd.Index(df_ledger), drop=True, inplace=True)
    
    # append df to df_list and df_list to your lists of dfs list
        df_list.append(df)
    lists_of_df_lists.append(df_list)


In [22]:
# this list will hold your merged data frames that will be exported to excel
list_of_merged_dfs = []

# this goes thought you lists of df list grabs every individual df_list
# inside and merges them all to gether.
for x in range(len(lists_of_df_lists)):
    df_static = lists_of_df_lists[x][0]

    for i in range(len(lists_of_df_lists[x])-1):
        temp_df1 = lists_of_df_lists[x][i]
        temp_df2 = lists_of_df_lists[x][i+1]

        cols_to_use = temp_df2.columns.difference(temp_df1.columns)

        temp_df2 = temp_df2[cols_to_use]  

        df_static = df_static.join(temp_df2,  how='left', rsuffix='_y')
    
    list_of_merged_dfs.append(df_static)


In [23]:
# this is just for nameing your excel sheets
list_of_financials = ['IncomeStatement','BalanceSheet','CashFlows']

In [24]:
# specify where you would like to save your file
save_to_path = r"C:\Users\{insert Username}\OneDrive\Desktop"

# specify file name
excel_file_name = r'\{insert file name}.xlsx'

# create full path
full_path = save_to_path + excel_file_name

In [25]:
# put all dfs into excel
list_of_merged_dfs[0].to_excel(full_path, sheet_name=list_of_financials[0])

book = load_workbook(full_path)
writer = pd.ExcelWriter(full_path, engine='openpyxl')
writer.book = book

list_of_merged_dfs[1].to_excel(writer, sheet_name=list_of_financials[1])
list_of_merged_dfs[2].to_excel(writer, sheet_name=list_of_financials[2])

writer.save()
writer.close()